In [1]:
from DB import *
db_connection = connect_db()

In [2]:
main = load_data_from_rds('생태통로', db_connection)
main

,번호,위도,경도
0,1495,35.36019,128.06375
1,1277,37.03408,128.40661
2,1278,36.89832,128.30563
3,1279,36.89687,128.30581
4,1090,36.71051,127.42222
...,...,...,...
492,1089,36.71278,127.42056
493,1087,36.71722,127.41778
494,1088,36.71222,127.42528
495,1091,36.76750,127.47333


In [7]:
# 경지계 데이터가 커 두 개로 나눠져 있음
farm1 = pd.read_csv('../data/경지계1-3.csv')
farm2 = pd.read_csv('../data/경지계4-7.csv')

In [8]:
farm1.head(3)

,번호,위도_십진,경도_십진,생태통로타,차선_수,등산객이용,생태통로효,이용확인종,이용확인,면적_m^2,너비_m,길이_m,join_UFID,join_DIVI,join_SCLS,join_FMTA,join_layer,join_path,distance
0,1495,35.36019,128.06375,0,1,1,3,NaN,0,NaN,NaN,NaN,VGT0100000037M3BF,FLD002,NaN,NaN,NGII_CDM_寃쎌�怨�3 �� Z_NGII_N3A_D0010000_C.shp,/vsizip/C:/Users/tkfkd/Downloads/NGII_CDM_寃쎌�怨...,20874.160929
1,1277,37.03408,128.40661,1,0,0,3,NaN,0,NaN,NaN,NaN,VGT010000000CPZX2,FLD002,NaN,NaN,NGII_CDM_寃쎌�怨�1 �� Z_NGII_N3A_D0010000_A.shp,/vsizip/C:/Users/tkfkd/Downloads/NGII_CDM_寃쎌�怨...,303.690012
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,VGT010000000R9IM9,FLD002,NaN,NaN,NGII_CDM_寃쎌�怨�1 �� Z_NGII_N3A_D0010000_A.shp,/vsizip/C:/Users/tkfkd/Downloads/NGII_CDM_寃쎌�怨...,11.153787


In [9]:
# 필요 없는 컬럼 drop
farm1.drop(['join_UFID',	'join_DIVI',	'join_SCLS',	'join_FMTA',	'join_layer',	'join_path'], axis=1, inplace=True)
farm2.drop(['join_UFID',	'join_DIVI',	'join_SCLS',	'join_FMTA',	'join_layer',	'join_path'], axis=1, inplace=True)

In [10]:
# 농가(경지계)까지의 최단 거리 저장 - farm1, farm2 두 데이터의 거리를 비교해 더 작은 것을 저장
for i in range(len(farm1)):
  dist_km = min(farm1['distance'][i], farm2['distance'][i]) / 1000 # 기존 distance는 미터 단위이므로 킬로미터 단위로 바꾸어 저장
  main.loc[i, '농가까지의 거리(km)'] = dist_km

In [12]:
main

,번호,위도,경도,농가까지의 거리(km)
0,1495,35.36019,128.06375,0.223706
1,1277,37.03408,128.40661,0.303690
2,1278,36.89832,128.30563,0.011154
3,1279,36.89687,128.30581,0.012767
4,1090,36.71051,127.42222,0.183505
...,...,...,...,...
492,1089,36.71278,127.42056,0.287976
493,1087,36.71722,127.41778,0.119958
494,1088,36.71222,127.42528,0.116240
495,1091,36.76750,127.47333,0.023978


## 최종 데이터

In [13]:
df = main[['번호', '농가까지의 거리(km)']]
df

,번호,농가까지의 거리(km)
0,1495,0.223706
1,1277,0.303690
2,1278,0.011154
3,1279,0.012767
4,1090,0.183505
...,...,...
492,1089,0.287976
493,1087,0.119958
494,1088,0.116240
495,1091,0.023978


## RDS에 데이터 업로드

In [14]:
table = load_data_from_rds('변수관계설정', db_connection)
table

,번호,등산로까지 최단거리(km),식생,경사도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도,주변 로드킬 빈도,산책로까지의 최단 거리(km),최고제한속도(km/h)
0,1001,1.903039,0,6,X,NaN,NaN,0,237.040073,80.0
1,1002,0.120319,1,5,X,4.0,4.0,0,122.055124,60.0
2,1004,0.047211,1,2,산지,1.0,1.0,0,125.516218,60.0
3,1005,0.233820,1,2,X,4.0,4.0,0,128.701343,80.0
4,1006,2.172428,0,4,하천,3.0,3.0,0,103.572133,80.0
...,...,...,...,...,...,...,...,...,...,...
492,1561,0.000416,1,2,산지,3.0,5.0,4,219.634740,60.0
493,1562,2.835246,1,5,산지,6.0,27.0,0,38.432601,80.0
494,1564,0.886170,1,4,X,1.0,1.0,0,248.086515,80.0
495,1567,9.125829,1,5,하천,4.0,5.0,0,283.374921,60.0


In [15]:
table = table.merge(df, on='번호')
table

,번호,등산로까지 최단거리(km),식생,경사도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도,주변 로드킬 빈도,산책로까지의 최단 거리(km),최고제한속도(km/h),농가까지의 거리(km)
0,1001,1.903039,0,6,X,NaN,NaN,0,237.040073,80.0,0.125686
1,1002,0.120319,1,5,X,4.0,4.0,0,122.055124,60.0,0.074627
2,1004,0.047211,1,2,산지,1.0,1.0,0,125.516218,60.0,0.110617
3,1005,0.233820,1,2,X,4.0,4.0,0,128.701343,80.0,0.009030
4,1006,2.172428,0,4,하천,3.0,3.0,0,103.572133,80.0,0.098269
...,...,...,...,...,...,...,...,...,...,...,...
492,1561,0.000416,1,2,산지,3.0,5.0,4,219.634740,60.0,0.087448
493,1562,2.835246,1,5,산지,6.0,27.0,0,38.432601,80.0,0.137822
494,1564,0.886170,1,4,X,1.0,1.0,0,248.086515,80.0,0.054073
495,1567,9.125829,1,5,하천,4.0,5.0,0,283.374921,60.0,0.102655


In [17]:
upload_data_to_rds(table, '변수관계설정', db_connection)